#PROYECTO FINAL HENRY
---
CASSANDRA

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Librerias Requeridas
!pip install cassandra-driver

from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## CONEXIÓN AL CLUSTER
  #El servidor no tiene cintraseña ni usuario para facilidad
cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory #Transforma la respuesta del cluster a dicionario

### Estructura de la base de datos

Las bases de datos de cassandra se dividen en las siguientes estructuras:

![Estructura Cassandra](https://www.red-gate.com/simple-talk/wp-content/uploads/2019/01/cassandra-cluster-node-l-n-keyspace-1-column-f.png "a title")|

* Keyspace = Database

Para este caso, el servidor se encuentra configurado con un solo nodo y el keyspace se llama 'henry'.

El Keyspace en este momento tiene dos (2) tablas, las cuales se estructuran de la siguiente manera:

* reviews
  * gmap_id: Identificación única del negocio.
  * state: Estado donde se encuentra ubicado el negocio.
  * user_id: Identificación única del usuario que realizó la reseña.
  * name: Nombre del usuario que realizó la reseña.
  * time: Fecha y hora en formato datetime.
  * rating: Calificación del usuario ( 1 a 5) como entero.
  * text: Texto del review realizado por el cliente.
  * resp: Respuesta al comentario del cliente como diccionario en *formato texto*. Incluye "time" como la fecha y hora en la que se repondió al comentario y "text" como la respuesta del comentario.

* stores
  * gmap_id: Identificación única del negocio.
  * name: Nombre del negocio.
  * address: Dirección completa del negocio.
  * category: Set que contiene las categorías a las que pertenece el restaurante. Se puede utilizar como filtro.
  * description:
  * hours:
  * latitud:
  * longitud:
  * price:
  *

In [ ]:
#for row in session.execute("SELECT * FROM stores WHERE category CONTAINS 'Restaurant' "):
#  print(row)

In [ ]:
""" categoria = "French restaurant"
statement = SimpleStatement("SELECT * FROM stores WHERE category CONTAINS %s;", fetch_size = 5000)
resp = session.execute(statement, (categoria, ), timeout=None)

dictionary = {'id':[], 'name': [], 'category': []}

for row in resp:
    dictionary['id'].append(row['gmap_id'])
    dictionary['name'].append(row['name'])
    dictionary['category'].append(row['category'])

stores = pd.DataFrame(dictionary) """

In [ ]:
# stores.head()

,id,name,category
0,0x808506614c768671:0xde51b62058db3bce,La Toque,"(French restaurant, Restaurant)"
1,0x80c2beaee2d80c53:0x63ead0297a8d12e,Le Relais De l'Entrecôte,"(Bistro, French restaurant, Steak house)"
2,0x89b38626e82d0729:0x9dd93adfaf7fc831,Water Street,"(American restaurant, French restaurant)"
3,0x80c2b1f48b8681e9:0xd8d4a07a5cb74352,Cafe Boheme,(French restaurant)
4,0x89c25989a1ed3b8d:0x667d317dad3afd1f,Parigot,(French restaurant)


# Reviews

In [ ]:
statement = SimpleStatement("SELECT * FROM reviews WHERE state IN ('Florida', 'New York', 'California', 'Nevada', 'Texas') ALLOW FILTERING;", fetch_size = 5000)
resp = session.execute(statement, timeout=None)
counter = 0

dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}


for row in resp:
    dictionary['gmap_id'].append(row['gmap_id'])
    dictionary['state'].append(row['state'])
    dictionary['user_id'].append(row['user_id'])
    dictionary['name'].append(row['name'])
    dictionary['time'].append(row['time'])
    dictionary['rating'].append(row['rating'])
    dictionary['text'].append(row['text'])
    dictionary['resp'].append(row['resp'])


reviews = pd.DataFrame(dictionary)
estados = reviews['state'].unique()

print('Rows Count: ' + str(len(reviews)) + '\n States Count : ' + str(len(reviews['state'].unique())))
print('States loaded:')
print(estados)

In [ ]:
reviews.info()

In [ ]:
reviews.head()

In [ ]:
valores_nulos = reviews[reviews['text'].isnull()]

# Mostrar los valores nulos
valores_nulos.count()

In [ ]:
reviews_final = reviews.dropna(subset=['text'])

In [ ]:
reviews_final.info()

In [ ]:
# Outliers in 'rating'
sns.boxplot(x='rating', data = reviews_final)
plt.show()

In [ ]:
reviews_final.duplicated().sum()


## Tabla 'Estados'

In [ ]:
estados

In [ ]:
df_estados = pd.DataFrame({'State': estados})
df_estados['State_id'] = range(1, len(estados) + 1)

In [ ]:
df_estados

## Tabla 'Cliente'

In [ ]:
df_clientes = reviews_final[['user_id', 'name']]
df_clientes = df_clientes.rename(columns={'user_id':'Client_id'})

In [ ]:
df_clientes.head(2)

## Tabla 'Review'

In [16]:
df_review = reviews_final[['user_id', 'gmap_id', 'time', 'rating', 'text']]

df_review = df_review.rename(columns={'time':'date'})
df_review = df_review.rename(columns={'user_id':'Client_id'})

df_review['Review_id'] = range(1, len(df_review) + 1)

In [17]:
df_review.head(2)

,Client_id,gmap_id,date,rating,text,Review_id
0,1.179187e+20,0x80c335e5afce634b:0xd412bd31f33038cd,2020-12-06 20:52:54.072,5,The place was great .. Our agent Trina is just...,1
1,1.177612e+20,0x80c335e5afce634b:0xd412bd31f33038cd,2019-06-05 22:35:02.670,5,Very supportive environment for sellers or buy...,2


Data Incremental (Ejemplo):

*Contador para el siguiente ID
next_id = max(df['IDUsuario']) + 1

*Nuevos datos incrementales
new_data = [{'Nombre':'Laura', 'Edad':33}]
new_df = pd.DataFrame(new_data)

*Asignar nuevo ID incremental
new_df['IDUsuario'] = range(next_id, next_id+len(new_df))

*Actualizar contador
next_id = max(new_df['IDUsuario']) + 1

*Unir con datos existentes
df = pd.concat([df, new_df]).reset_index(drop=True)

## Tabla 'Comentarios'

In [ ]:
df_review_2 = df_review.rename(columns={'Client_id':'user_id'})
reviews_final_2 = reviews_final.dropna(subset=['resp'])
merged = reviews_final_2.merge(df_review_2[['user_id', 'date', 'Review_id']], on='user_id')

In [ ]:
merged.head(1)

In [ ]:
df_comentarios = merged[['Review_id','resp', 'date']]
df_comentarios['Comment_id'] = range(1, len(df_comentarios) + 1)

In [ ]:

reviews_final[['time', 'resp']]

df_comentarios = df_review[['Review_id', 'date']]

df_review = df_review.rename(columns={'time':'date'})
df_review = df_review.rename(columns={'user_id':'Client_id'})

df_review['Review_id'] = range(1, len(df_review) + 1)

# Stores

In [25]:
for row in session.execute("SELECT * FROM stores  LIMIT 5"):
  print(row)

{'gmap_id': '0x89c260cbcf2848c7:0x9b12a43601b1da0d', 'name': 'Chandrabali Home Improvement', 'address': 'Chandrabali Home Improvement, 133-20 95th Ave, Queens, NY 11419', 'category': SortedSet(['General contractor']), 'description': None, 'hours': "[['Monday', '7AM–5PM'], ['Tuesday', '7AM–5PM'], ['Wednesday', '7AM–5PM'], ['Thursday', '7AM–5PM'], ['Friday', '7AM–5PM'], ['Saturday', '8AM–1PM'], ['Sunday', 'Closed']]", 'latitude': 40.695640563964844, 'longitude': -73.81490325927734, 'misc': 'None', 'price': 'None', 'url': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c260cbcf2848c7:0x9b12a43601b1da0d?authuser=-1&hl=en&gl=us'}
{'gmap_id': '0x88f67d6459172897:0xb6897aae90060770', 'name': 'Morgan County Detention Center', 'address': 'Morgan County Detention Center, 1380 Monticello Rd, Madison, GA 30650', 'category': SortedSet(['Government office']), 'description': None, 'hours': "[['Wednesday', 'Open 24 hours'], ['Thursday', 'Open 24 hours'], ['Friday', 'Open 24 hours'], ['Saturday

In [33]:
categories_searched = ['Restaurant',
                       'restaurant',
                       'Bar',
                       'bar',
                       'Deli',
                       'Grocery',
                       'Coffee',
                       'Bakery',
                       'Sandwich']

dictionary = {'gmap_id':[],
              'name': [],
              'address': [],
              'latitude': [],
              'longitude': [],
              'category':[],
              'misc':[]}


for category_searched in categories_searched:
  # Construct the query
  query = f"""
        SELECT gmap_id,name,address,latitude,longitude,category, misc
        FROM stores
        WHERE category CONTAINS '{category_searched}'
        """
  statement = SimpleStatement(query, fetch_size = 3000)

  answer = session.execute(statement, timeout=None)

  for row in answer:
    dictionary['gmap_id'].append(row['gmap_id'])
    dictionary['name'].append(row['name'])
    dictionary['address'].append(row['address'])
    dictionary['latitude'].append(row['latitude'])
    dictionary['longitude'].append(row['longitude'])
    dictionary['category'].append(row['category'])
    dictionary['misc'].append(row['misc'])

In [34]:
# Create a DataFrame from the dictionary
df_stores = pd.DataFrame(dictionary)

df_stores.head()

,gmap_id,name,address,latitude,longitude,category,misc
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"(ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...
1,0x80c8bf86a8b9a3f1:0xacfe57cd1919e249,Ember's Grill + Spirits,"Ember's Grill + Spirits, 740 S Rampart Blvd, L...",36.163380,-115.289413,"(Bar, Restaurant)","{'Service options': ['Delivery'], 'Highlights'..."
2,0x808506614c768671:0xde51b62058db3bce,La Toque,"La Toque, 1314 McKinstry St, Napa, CA 94559",38.304173,-122.283989,"(French restaurant, Restaurant)","{'Service options': ['Outdoor seating', 'Dine-..."
3,0x8842ecec5931b86b:0x703640198411596d,Godfather's Pizza,"Godfather's Pizza, 233 Lexington Rd, Lancaster...",37.625710,-84.580132,"(Buffet restaurant, Delivery Restaurant, Itali...","{'Service options': ['Delivery'], 'Popular for..."
4,0x87e030ec559cac7f:0x8f04ab05959695e6,Subway,"Subway, 330 N 1st St, Cuba, IL 61427",40.496239,-90.198875,"(Caterer, Fast food restaurant, Restaurant, Sa...","{'Service options': ['Curbside pickup', 'Takeo..."


In [35]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157444 entries, 0 to 157443
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   gmap_id    157444 non-null  object 
 1   name       157444 non-null  object 
 2   address    156787 non-null  object 
 3   latitude   157444 non-null  float64
 4   longitude  157444 non-null  float64
 5   category   157444 non-null  object 
 6   misc       157444 non-null  object 
dtypes: float64(2), object(5)
memory usage: 8.4+ MB
